# 99 Scala Exercises (46 to 50)

### 46\. Truth tables for logical expressions.
Define functions and, or, nand, nor, xor, impl, and equ (for logical equivalence) which return true or false according to the result of their respective operations; e.g. and(A, B) is true if and only if both A and B are true.

In [4]:
import scalaz.effect.IO._
import scalaz.effect.IO

def and(a: Boolean, b: Boolean): Boolean = a && b
def or(a: Boolean, b: Boolean): Boolean = a || b
def xor(a: Boolean, b: Boolean): Boolean = a ^ b
def nand(a: Boolean, b: Boolean): Boolean = !and(a, b)
def nor(a: Boolean, b: Boolean): Boolean = !or(a, b)
def impl(a: Boolean, b: Boolean): Boolean = !a || b
def equ(a: Boolean, b: Boolean): Boolean = a == b
def not(a: Boolean): Boolean = !a

def table2(f: (Boolean, Boolean) => Boolean): IO[Unit] = for {
    _ <- putStrLn("A       B      result")
    _ <- putStrLn(s"True    True   ${ f(true, true) }")
    _ <- putStrLn(s"True    False  ${ f(true, false) }")
    _ <- putStrLn(s"False   True   ${ f(false, true) }")
    _ <- putStrLn(s"False   False  ${ f(false, false) }")
} yield ()

table2(or).unsafePerformIO

A       B      result
True    True   true
True    False  true
False   True   true
False   False  false


import scalaz.effect.IO._

import scalaz.effect.IO


defined function and
defined function or
defined function xor
defined function nand
defined function nor
defined function impl
defined function equ
defined function not
defined function table2

### 47\. Truth tables for logical expressions (2).
Continue problem P46 by redefining and, or, etc as operators. (i.e. make them methods of a new class with an implicit conversion from Boolean.) not will have to be left as a object method.

In [7]:
implicit class BooleanOps(val a: Boolean) {
    def not(): Boolean = !a
    def and(b: Boolean): Boolean = a && b
    def or(b: Boolean): Boolean = a || b
    def xor(b: Boolean): Boolean = a ^ b
    def nand(b: Boolean): Boolean = !(a and b)
    def nor(b: Boolean): Boolean = !(a or b)
    def impl(b: Boolean): Boolean = !a || b
    def equ(b: Boolean): Boolean = a == b
}

table2((a: Boolean, b: Boolean) => a and (a or not(b))).unsafePerformIO

A       B      result
True    True   true
True    False  true
False   True   false
False   False  false


defined class BooleanOps

### 49\. Gray code.
An n-bit Gray code is a sequence of n-bit strings constructed according to certain rules. For example,
n = 1: C(1) = ("0", "1").
n = 2: C(2) = ("00", "01", "11", "10").
n = 3: C(3) = ("000", "001", "011", "010", "110", "111", "101", "100").

Find out the construction rules and write a function to generate Gray codes.

In [25]:
// Function wrapper that allow memoization.
case class Memoize[A,B](val f: A => B) extends (A => B) {
  val cache = collection.mutable.HashMap[A,B]()
  def apply(a: A): B = cache getOrElseUpdate(a, f(a))
}

val gray: Memoize[Int, List[String]] = Memoize { n =>
    n match {
        case 2 => List("00", "01", "11", "10")
        case _ => {
            val g = gray(n-1)
            g.map("0"+_) ++ g.reverse.map("1"+_)
        }
    }
}

gray(3)

defined class Memoize
gray: wrapper.wrapper.Memoize[Int, List[String]] = <function1>
res24_2: List[String] = List("000", "001", "011", "010", "110", "111", "101", "100")

### Huffman code.
First of all, consult a good book on discrete mathematics or algorithms for a detailed description of Huffman codes!
We suppose a set of symbols with their frequencies, given as a list of (S, F) Tuples. E.g. (("a", 45), ("b", 13), ("c", 12), ("d", 16), ("e", 9), ("f", 5)). Our objective is to construct a list of (S, C) Tuples, where C is the Huffman code word for the symbol S.

In [39]:
// Simple binary tree.
sealed trait HTree {
    def weight(): Int = this match {
        case HLeaf(_, f)    => f
        case HFork(s, _, _) => s
    }
    
    def merge(other: HTree): HTree =
        HFork(this.weight + other.weight, this, other)
}

final case class HLeaf(s: String, freq: Int) extends HTree
final case class HFork(sum: Int, l: HTree, r: HTree) extends HTree

implicit object HTreeOrdering extends Ordering[HTree] {
    def compare(a: HTree, b: HTree): Int = a.weight compare b.weight
}

// Insert element in a sorted list.
def insertBy[A](l: List[A], x: A)(implicit ord: Ordering[A]): List[A] = {
    val (first, last) = l partition { ord.lteq(_, x) }
    first:::x::last
}

def buildTree(sl: List[(String, Int)]): Option[HTree] = {
    val nl = sl.sortBy(_._2) map { case (s, f) => HLeaf(s, f) }
    @annotation.tailrec
    def inner(accum: List[HTree]): Option[HTree] = accum match {
        case x::y::xs => inner(insertBy(xs, x merge y))
        case x::Nil   => Some(x)
        case _        => None
    }
    inner(nl)
}

def codify(tree: HTree): List[(String, String)] = {
    def inner(tree: HTree, prefix: String): List[(String, String)] = tree match {
        case HLeaf(s, _)    => List((s, prefix))
        case HFork(_, l, r) => inner(l, prefix + "0") ++ inner(r, prefix + "1")
    }
    inner(tree, "")
}

def huffman(sl: List[(String, Int)]): Option[List[(String, String)]] =
    buildTree(sl) map(codify(_).sortBy(_._1))

huffman(List(("a", 45), ("b", 13), ("c", 12), ("d", 16), ("e", 9), ("f", 5)))

defined trait HTree
defined class HLeaf
defined class HFork
defined object HTreeOrdering
defined function insertBy
defined function buildTree
defined function codify
defined function huffman
res38_8: Option[List[(String, String)]] = Some(
  List(
    ("a", "0"),
    ("b", "101"),
    ("c", "100"),
    ("d", "111"),
    ("e", "1101"),
    ("f", "1100")
  )
)